In [0]:
print("Welcome Databricks")

In [0]:
%sql
select * from default

In [0]:
dbutils.fs.help()

In [0]:
%sql
USE CATALOG hive_metastore

In [0]:
%sql
CREATE TABLE employees
  (id INT, name STRING, salary DOUBLE);

In [0]:
%sql
INSERT INTO employees
VALUES 
  (1, "vaibhav", 500.0),
  (2, "vps", 600.5);

INSERT INTO employees
VALUES
  (3, "harshit", 999.3),
  (4, "shishir", 400.3);

INSERT INTO employees
VALUES
  (5, "Aman", 1500.0);

INSERT INTO employees
VALUES
  (6, "Rajesh", 1200.3)


In [0]:
%sql
SELECT * FROM employees

In [0]:
%sql
DESCRIBE DETAIL employees


In [0]:
%fs
ls 'dbfs:/user/hive/warehouse/employees'

In [0]:
%sql
UPDATE employees 
SET salary = salary + 100
WHERE name LIKE "v%"

In [0]:
%sql
DESCRIBE DETAIL employees

In [0]:
%sql
DESCRIBE HISTORY employees

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'

In [0]:
%fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000005.json'

In [0]:
%sql
SELECT * 
FROM employees VERSION AS OF 4

In [0]:
%sql
SELECT * FROM employees@v4

In [0]:
%sql
DELETE FROM employees

In [0]:
%sql
SELECT * FROM employees

In [0]:
%sql
DESCRIBE HISTORY employees

In [0]:
%sql
VACUUM employees

In [0]:
%sql
VACUUM employees RETAIN 0 HOURS

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false

In [0]:
%sql
DROP TABLE employees

In [0]:
name = "ananth"

In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS ananth2databricsstorageEXT
    URL 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/'
    WITH (STORAGE CREDENTIAL akallidatabricscredentials)
    COMMENT 'External Location for Db Training'

In [0]:
%sql
SHOW CATALOGS

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS organizationdata
      MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/'
      COMMENT 'This is the catalog for the Organization Data' ;

In [0]:
%sql
SELECT current_catalog();

In [0]:
%sql
USE CATALOG organizationdata;


CREATE SCHEMA IF NOT EXISTS landing
     MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing';  
CREATE SCHEMA IF NOT EXISTS bronze
     MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/bronze';  
CREATE SCHEMA IF NOT EXISTS silver
     MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/silver';
CREATE SCHEMA IF NOT EXISTS gold
     MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/gold';


In [0]:
%sql
SHOW CATALOGS

In [0]:
%sql
USE CATALOG organizationdata;
USE SCHEMA landing;

CREATE EXTERNAL VOLUME IF NOT EXISTS operational_data
    LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/operational_data/';

In [0]:
%fs ls /Volumes/organizationdata/landing/operational_data/addresses

In [0]:
%sql
SELECT * FROM json.`/Volumes/organizationdata/landing/operational_data/customers/customers_2024_10.json`

In [0]:
%sql
SELECT * FROM json.`/Volumes/organizationdata/landing/operational_data/customers/customers_2024_*.json`

In [0]:
%sql
SELECT * FROM json.`/Volumes/organizationdata/landing/operational_data/customers`

In [0]:
%sql
SELECT input_file_name() AS file_path, 
       _metadata.file_path AS metadata_file_path,
       * 
  FROM json.`/Volumes/organizationdata/landing/operational_data/customers`

In [0]:
%sql
CREATE OR REPLACE VIEW organizationdata.bronze.v_customers
AS
SELECT *,
       _metadata.file_path AS file_path 
  FROM json.`/Volumes/organizationdata/landing/operational_data/customers`

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_customers
AS
SELECT *,
       _metadata.file_path AS file_path 
  FROM json.`/Volumes/organizationdata/landing/operational_data/customers`

In [0]:
%sql
SELECT * FROM tv_customers;

In [0]:
%sql
CREATE OR REPLACE GLOBAL TEMPORARY VIEW gtv_customers
AS
SELECT *,
       _metadata.file_path AS file_path 
  FROM json.`/Volumes/organizationdata/landing/operational_data/customers`

In [0]:
%sql
SELECT * FROM global_temp.gtv_customers

In [0]:
%sql
--Query Orders File using TEXT Format
SELECT * FROM text.`/Volumes/organizationdata/landing/operational_data/orders`;

In [0]:

%sql
SELECT * FROM binaryFile.`/Volumes/organizationdata/landing/operational_data/memberships/2024-10/*.png`

In [0]:
%sql
SELECT * FROM binaryFile.`/Volumes/organizationdata/landing/operational_data/memberships/*/*.png`

In [0]:
%sql
--Demonstrate Limitations of CSV file format in SELECT statement
SELECT * FROM csv.`/Volumes/organizationdata/landing/operational_data/addresses`

In [0]:
%sql
SELECT * 
  FROM read_files('/Volumes/organizationdata/landing/operational_data/addresses',
                  format => 'csv',
                  delimiter => '\t',
                  header => true);

In [0]:
%sql
CREATE OR REPLACE VIEW organizationdata.bronze.v_addresses
AS
SELECT * 
  FROM read_files('/Volumes/organizationdata/landing/operational_data/addresses',
                  format => 'csv',
                  delimiter => '\t',
                  header => true);

In [0]:
%fs ls 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/operational_data/payments/'

In [0]:
#Run SQL Commands using Python - spark.sql Function
df = spark.sql('SELECT * FROM json.`/Volumes/organizationdata/landing/operational_data/customers`')
display(df)

In [0]:
df = spark.sql('''CREATE OR REPLACE TEMP VIEW tv_customers 
                  AS 
                  SELECT * FROM json.`/Volumes/organizationdata/landing/operational_data/customers`''')

In [0]:
%sql
SELECT * FROM tv_customers;

In [0]:
#Spark DataframeReader API
df = spark.read.format('json').load('/Volumes/organizationdata/landing/operational_data/customers')
display(df)

In [0]:
#spark.table Function
df = spark.table('organizationdata.bronze.v_addresses')
display(df)

Databricks data profile. Run in Databricks to view.

In [0]:
%python 
df = spark.table('organizationdata.bronze.v_customers')
dbutils.data.summarize(df)

In [0]:
# Read files using DataStreamReader API

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

customers_schema = StructType(fields=[StructField("customer_id", IntegerType()),
                                     StructField("customer_name", StringType()),
                                     StructField("date_of_birth", DateType()),
                                     StructField("telephone", StringType()),
                                     StructField("email", StringType()),
                                     StructField("member_since", DateType()),
                                     StructField("created_timestamp", TimestampType())
                                    ]
                              )

In [0]:
customers_df = (
                    spark.readStream
                         .format("json")
                         .schema(customers_schema)
                         .load("/Volumes/organizationdata/landing/operational_data/customers_stream/")
)

In [0]:
#Transform the dataframe to add the following columns
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (
    customers_df
    .withColumn("file_path", col("_metadata.file_path"))
    .withColumn("ingestion_date", current_timestamp())
)

In [0]:
# Write the transformed data stream to Delta Table

streaming_query = (
    customers_transformed_df.writeStream
        .format("delta")
        .option("checkpointLocation", "/Volumes/organizationdata/landing/operational_data/customers_stream/_checkpoint_stream")
        .toTable("organizationdata.bronze.customers_stream")
)

#streaming_query.stop() --to interupt the stream

In [0]:
%sql
SELECT * FROM organizationdata.bronze.customers_stream;

In [0]:
# Write the transformed data stream to Delta Table

streaming_query = (
    customers_transformed_df.writeStream
        .format("delta")
        .option("checkpointLocation", "/Volumes/organizationdata/landing/operational_data/customers_stream/_checkpoint_stream")
        .toTable("organizationdata.bronze.customers_stream")
) .outputMode("complete")

In [0]:
customers_df = (
                    spark.readStream
                         .format("cloudFiles")
                         .option("cloudFiles.format", "json")
                         .option("cloudFiles.schemaLocation", "/Volumes/organizationdata/landing/operational_data/customers_autoloader/_schema")
                         .option("cloudFiles.inferColumnTypes", "true")
                         .option("cloudFiles.schemaHints", "date_of_birth DATE, member_since DATE, created_timestamp TIMESTAMP")
                         .load("/Volumes/organizationdata/landing/operational_data/customers_autoloader/") 
                )

In [0]:
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (
    customers_df.withColumn("file_path", col("_metadata.file_path"))
                .withColumn("ingestion_date", current_timestamp())
)

display(customers_transformed_df)

In [0]:
streaming_query = (
    customers_transformed_df.writeStream
        .format("delta")
        .option("checkpointLocation", "/Volumes/organizationdata/landing/operational_data/customers_autoloader/_checkpoint_stream")
        .start("organizationdata.bronze.customers_autoloader")
)

In [0]:
streaming_query.stop()

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS delta_lake
    MANAGED LOCATION 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/dlt_advanced';

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.companies;
CREATE TABLE delta_lake.companies
  (company_name STRING,
   founded_date DATE,
   country      STRING)
COMMENT 'This table contains information about some of the successful tech companies'   
TBLPROPERTIES ('sensitive' = 'true', 'delta.enableDeletionVectors' = 'false');

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.companies;
CREATE TABLE delta_lake.companies
  (company_name STRING NOT NULL,
   founded_date DATE COMMENT 'The date the company was founded',
   country      STRING)
COMMENT 'This table contains information about some of the successful tech companies'   
TBLPROPERTIES ('sensitive' = 'true', 'delta.enableDeletionVectors' = 'false');

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.companies;
CREATE TABLE delta_lake.companies
  (company_id BIGINT NOT NULL GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
   company_name STRING NOT NULL,
   founded_date DATE COMMENT 'The date the company was founded',
   country      STRING)
COMMENT 'This table contains information about some of the successful tech companies'   
TBLPROPERTIES ('sensitive' = 'true', 'delta.enableDeletionVectors' = 'false');

In [0]:
%sql
INSERT INTO delta_lake.companies 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "INDIA"),
       ("TCS", "1975-04-04", "INDIA"),
       ("wipro", "1998-09-04", "INDIA"),
       ("Infosys", "1994-07-05", "INDIA");

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.companies;

CREATE TABLE delta_lake.companies
  (company_id BIGINT NOT NULL GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
   company_name STRING,
   founded_date DATE,
   country      STRING);

INSERT INTO delta_lake.companies 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "INDIA"),
       ("TCS", "1975-04-04", "INDIA"),
       ("wipro", "1998-09-04", "INDIA"),
       ("Infosys", "1994-07-05", "INDIA"),
("HUL","1996-02-11","USA")

In [0]:
%sql
DESC HISTORY delta_lake.companies;

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.companies;

In [0]:
%sql
CREATE OR REPLACE TABLE delta_lake.companies
  (company_id BIGINT NOT NULL GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
   company_name STRING,
   founded_date DATE,
   country      STRING);

INSERT INTO delta_lake.companies 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "INDIA"),
       ("TCS", "1975-04-04", "INDIA"),
       ("wipro", "1998-09-04", "INDIA"),
       ("Infosys", "1994-07-05", "INDIA"),
       ("HUL","1996-02-11","USA");

In [0]:
%sql
DESC HISTORY delta_lake.companies;

In [0]:
%sql
CREATE TABLE delta_lake.companies_USA
AS
SELECT CAST(company_id AS INT) AS company_id,
       company_name,
       founded_date,
       country 
  FROM delta_lake.companies
 WHERE country = 'USA';

In [0]:
%sql
DESC delta_lake.companies_USA;

In [0]:
%sql
ALTER TABLE delta_lake.companies_USA 
  ALTER COLUMN founded_date COMMENT 'Date the company was founded';

In [0]:
%sql
ALTER TABLE delta_lake.companies_USA 
  ALTER COLUMN company_id SET NOT NULL;

In [0]:
%sql
SELECT * FROM delta_lake.companies_USA;


In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.gold_companies;

CREATE TABLE delta_lake.gold_companies
  (company_name STRING,
   founded_date DATE,
   country      STRING);

INSERT INTO delta_lake.gold_companies 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "USA"),  
       ("HUL", "1998-11-11", "China"); 

SELECT * FROM delta_lake.gold_companies;

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.bronze_companies;

CREATE TABLE delta_lake.bronze_companies
  (company_name STRING,
   founded_date DATE,
   country      STRING);

INSERT INTO delta_lake.bronze_companies 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "INDIA"),
       ("TCS", "1975-04-04", "INDIA"),
       ("wipro", "1998-09-04", "INDIA"),
       ("Infosys", "1994-07-05", "INDIA"),
       ("HUL","1996-02-11","USA");  

SELECT * FROM delta_lake.bronze_companies;

In [0]:
%sql
INSERT OVERWRITE TABLE delta_lake.gold_companies --schema should be same to do this overwrite
SELECT *
  FROM delta_lake.bronze_companies;

In [0]:
%sql
SELECT * FROM delta_lake.gold_companies;

In [0]:
%sql
DESC HISTORY delta_lake.gold_companies;

In [0]:
%sql
DROP TABLE IF EXISTS delta_lake.gold_companies_partitioned;

CREATE TABLE delta_lake.gold_companies_partitioned
  (company_name STRING,
   founded_date DATE,
   country      STRING)
PARTITIONED BY (country);

INSERT INTO delta_lake.gold_companies_partitioned 
(company_name, founded_date, country)
VALUES ("Reliance", "1976-04-01", "INDIA"),  
       ("HUL", "1998-11-11", "USA"); 

SELECT * FROM delta_lake.gold_companies_partitioned;

In [0]:
%sql
DESC EXTENDED delta_lake.gold_companies_partitioned

In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta_lake.raw_stock_prices; 

In [0]:
%sql
COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true');


In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

In [0]:
%sql
DELETE FROM delta_lake.raw_stock_prices;

COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
SELECT * FROM delta_lake.raw_stock_prices;

In [0]:
%sql
DELETE FROM delta_lake.raw_stock_prices;

COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
SELECT * FROM delta_lake.raw_stock_prices;

In [0]:
%sql
create table delta_lake.stock_prices;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta_lake.raw_stock_prices;

In [0]:
%sql
use catalog organizationdata;
create schema if not exists delta_lake

In [0]:
%sql
USE SCHEMA delta_lake;
CREATE TABLE IF NOT EXISTS delta_lake.raw_stock_prices;

In [0]:
%sql
COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
select * from delta_lake.raw_stock_prices

In [0]:
%sql
COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
select * from delta_lake.raw_stock_prices

In [0]:
%sql
--Merge Statement
CREATE TABLE IF NOT EXISTS delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

In [0]:
%sql
MERGE INTO delta_lake.stock_prices AS target
USING delta_lake.raw_stock_prices AS source
   ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
    UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN    
    DELETE
WHEN NOT MATCHED AND source.status ='ACTIVE' THEN     
    INSERT (stock_id, price, trading_date) VALUES (source.stock_id, source.price, source.trading_date);

In [0]:
%sql
SELECT * FROM delta_lake.stock_prices;

In [0]:
%sql
DELETE FROM delta_lake.raw_stock_prices;

COPY INTO delta_lake.raw_stock_prices
FROM 'abfss://organizationdata@ananth2databricsstorage.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
MERGE INTO delta_lake.stock_prices AS target
USING delta_lake.raw_stock_prices AS source
   ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
    UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN    
    DELETE
WHEN NOT MATCHED AND source.status ='ACTIVE' THEN     
    INSERT (stock_id, price, trading_date) VALUES (source.stock_id, source.price, source.trading_date);


In [0]:
%sql
SELECT * FROM delta_lake.stock_prices;